In [ ]:
import os
import json

import numpy as np

from pymedphys.level1.msqconnect import mosaiq_connect
from pymedphys.level1.configutilities import get_index, get_data_directory
from pymedphys.level2.msqdelivery import multi_fetch_and_verify_mosaiq

from pymedphys.level4.comparebygantry import (
    group_consecutive_logfiles, assert_array_agreement, get_gantry_tolerance,
    get_logfile_delivery_data_bygantry, get_logfile_mu_density_bygantry,
    get_mosaiq_delivery_data_bygantry, get_mosaiq_mu_density_bygantry,
    get_comparison_results, get_mappings, get_comparisons_byfield
)

In [ ]:
with open('../config.json') as config_file:
    config = json.load(config_file)

In [ ]:
data_directory = get_data_directory(config)

cache_filepath = os.path.join(data_directory, 'cache', 'dmlc_comparison.json')
cache_scratch_filepath = os.path.join(data_directory, 'cache', 'dmlc_comparison_scratch.json')

with open(cache_filepath, 'r') as cache_file:
    cache = json.load(cache_file)

In [ ]:
index = get_index(config)
file_hashes = np.array(list(index.keys()))

field_types = np.array([
    index[file_hash]['delivery_details']['field_type']
    for file_hash in file_hashes
])

file_hashes = file_hashes[field_types == 'DMLC']

is_qa = np.array([
    index[file_hash]['delivery_details']['qa_mode']
    for file_hash in file_hashes
])

file_hashes = file_hashes[np.invert(is_qa)]

machine = np.array([
    index[file_hash]['logfile_header']['machine']
    for file_hash in file_hashes
])

# limit to RCCC for now
file_hashes = file_hashes[(machine == '2619') | (machine == '2694')]
np.random.shuffle(file_hashes)

# might need course information
index[file_hashes[0]]

In [ ]:
patient_grouped_fields, field_id_grouped_hashes = get_mappings(index, file_hashes)

In [ ]:
patient_ids = list(patient_grouped_fields.keys())

In [ ]:
patient_id = patient_ids[0]
field_ids = patient_grouped_fields[patient_id]

In [ ]:
patient_id

In [ ]:
# for field_id in field_ids:
#     print(field_id_grouped_hashes[field_id])

In [ ]:
def get_comparisons_byfield(index, config, field_ids, field_id_grouped_hashes):    
    mosaiq_delivery_data_byfield = dict()

    with mosaiq_connect('msqsql') as cursor:
        for field_id in field_ids:
            mosaiq_delivery_data_byfield[field_id] = multi_fetch_and_verify_mosaiq(
                cursor, field_id)

    comparisons_byfield = dict()

    for field_id in field_ids:
        keys = np.array(field_id_grouped_hashes[field_id])
        logfile_groups = group_consecutive_logfiles(keys, index)
        logfile_groups = [
            tuple(group)
            for group in logfile_groups
        ]

        mosaiq_delivery_data = mosaiq_delivery_data_byfield[field_id]
        mosaiq_gantry_angles = np.unique(mosaiq_delivery_data.gantry)

        logfile_delivery_data_bygantry = get_logfile_delivery_data_bygantry(
            index, config, logfile_groups, mosaiq_gantry_angles)
        logfile_mu_density_bygantry = get_logfile_mu_density_bygantry(
            logfile_groups, mosaiq_gantry_angles, logfile_delivery_data_bygantry)
        mosaiq_delivery_data_bygantry = get_mosaiq_delivery_data_bygantry(
            mosaiq_delivery_data)
        mosaiq_mu_density_bygantry = get_mosaiq_mu_density_bygantry(
            mosaiq_delivery_data_bygantry)
        comparison_results = get_comparison_results(
            mosaiq_mu_density_bygantry, logfile_mu_density_bygantry)

        comparisons_byfield[field_id] = comparison_results
    
    return comparisons_byfield
        
comparisons_byfield = get_comparisons_byfield(index, config, field_ids, field_id_grouped_hashes)

In [ ]:
comparisons_byfield